In [2]:
import logging
import os
import threadpoolctl
import torch
from pathlib import Path
from lightning_fabric import seed_everything

import numpy as np
import pandas as pd
from rectools import Columns

from implicit.als import AlternatingLeastSquares

from rectools.dataset import Dataset
from rectools.metrics import MAP, calc_metrics, MeanInvUserFreq, Serendipity
from rectools.models import ImplicitALSWrapperModel
from rectools.models import SASRecModel
from rectools.models.nn.item_net import CatFeaturesItemNet, IdEmbeddingsItemNet

In [3]:
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

# For implicit ALS
os.environ["OPENBLAS_NUM_THREADS"] = "1"
threadpoolctl.threadpool_limits(1, "blas")

logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)
logger = logging.getLogger()

# Data

In [4]:
# %%time
# !wget -q https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip -O data_original.zip
# !unzip -o data_original.zip
# !rm data_original.zip

In [5]:
DATA_PATH = Path("data_original")

interactions = (
    pd.read_csv(DATA_PATH / 'interactions.csv', parse_dates=["last_watch_dt"])
    .rename(columns={"last_watch_dt": "datetime"})
)

# Split dataset

In [6]:
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)

# Split to train / test
max_date = interactions[Columns.Datetime].max()
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()
train.drop(train.query("total_dur < 300").index, inplace=True)

# drop items with less than 20 interactions in train
items = train["item_id"].value_counts()
items = items[items >= 20]
items = items.index.to_list()
train = train[train["item_id"].isin(items)]
    
# drop users with less than 2 interactions in train
users = train["user_id"].value_counts()
users = users[users >= 2]
users = users.index.to_list()
train = train[(train["user_id"].isin(users))]

users = train["user_id"].drop_duplicates().to_list()

# drop cold users from test
test_users_sasrec = test[Columns.User].unique()
cold_users = set(test[Columns.User]) - set(train[Columns.User])
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)
test_users = test[Columns.User].unique()


In [7]:
items = pd.read_csv(DATA_PATH / 'items.csv')

In [8]:
# Process item features to the form of a flatten dataframe
items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()
items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"
item_features = pd.concat((genre_feature, content_feature))

candidate_items = interactions['item_id'].drop_duplicates().astype(int)
test["user_id"] = test["user_id"].astype(int)
test["item_id"] = test["item_id"].astype(int)

catalog=train[Columns.Item].unique()

In [9]:
dataset_no_features = Dataset.construct(
    interactions_df=train,
)

dataset_item_features = Dataset.construct(
    interactions_df=train,
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)

In [10]:
metrics_name = {
    'MAP': MAP,
    'MIUF': MeanInvUserFreq,
    'Serendipity': Serendipity
    

}
metrics = {}
for metric_name, metric in metrics_name.items():
    for k in (1, 5, 10):
        metrics[f'{metric_name}@{k}'] = metric(k=k)

# list with metrics results of all models
features_results = []

# SASRec

In [11]:
RANDOM_SEED = 32
torch.use_deterministic_algorithms(True)
seed_everything(RANDOM_SEED, workers=True)

Seed set to 32


32

## Softmax loss

In [12]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    item_net_block_types=(IdEmbeddingsItemNet, ),  # Use only item ids in ItemNetBlock
    recommend_device="cuda",
)


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [13]:
%%time
model.fit(dataset_no_features)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params
---------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 927 K 
---------------------------------------------------------------
927 K     Trainable params
0         Non-trainable params
927 K     Total params
3.709     Total estimated model params size (MB)
/data/home/maspirina1/tasks/repo/RecTools/.venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=143` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: user 5min 48s, sys: 16.2 s, total: 6min 4s
Wall time: 5min 51s


In [14]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_no_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)

/data/home/maspirina1/tasks/repo/RecTools/rectools/models/nn/transformer_data_preparator.py:271: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/maspirina1/tasks/repo/RecTools/rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/data/home/maspirina1/tasks/repo/RecTools/.venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `nu

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 30.4 s, sys: 4.66 s, total: 35 s
Wall time: 24.9 s


In [15]:
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)
metric_values["model"] = "softmax"
features_results.append(metric_values)


In [16]:
features_results

[{'MAP@1': 0.04914305298031172,
  'MAP@5': 0.08279258397599479,
  'MAP@10': 0.09181125748466501,
  'MIUF@1': 3.672515395335384,
  'MIUF@5': 4.330234457796629,
  'MIUF@10': 4.916948486795568,
  'Serendipity@1': 0.0008783295933624946,
  'Serendipity@5': 0.0007228737224204921,
  'Serendipity@10': 0.0006564757640765052,
  'model': 'softmax'}]

In [17]:
recos

,user_id,item_id,score,rank
0,73446,7793,1.994760,1
1,73446,3784,1.958904,2
2,73446,9728,1.813175,3
3,73446,7829,1.624941,4
4,73446,15266,1.510166,5
...,...,...,...,...
947045,857162,6809,2.263752,6
947046,857162,12995,2.047079,7
947047,857162,4151,1.961828,8
947048,857162,657,1.664796,9


## BCE loss

In [19]:
RANDOM_SEED = 32
torch.use_deterministic_algorithms(True)
seed_everything(RANDOM_SEED, workers=True)

Seed set to 32


32

In [20]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    loss="BCE",
    n_negatives=2,
    item_net_block_types=(IdEmbeddingsItemNet, )  # Use only item ids in ItemNetBlock
)


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [21]:
%%time
model.fit(dataset_no_features)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params
---------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 927 K 
---------------------------------------------------------------
927 K     Trainable params
0         Non-trainable params
927 K     Total params
3.709     Total estimated model params size (MB)
/data/home/maspirina1/tasks/repo/RecTools/.venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=143` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: user 6min 18s, sys: 10.9 s, total: 6min 29s
Wall time: 6min 19s


In [22]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_no_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn",
)

/data/home/maspirina1/tasks/repo/RecTools/rectools/models/nn/transformer_data_preparator.py:271: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/maspirina1/tasks/repo/RecTools/rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/data/home/maspirina1/tasks/repo/RecTools/.venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `nu

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 27.9 s, sys: 4.28 s, total: 32.2 s
Wall time: 23.3 s


In [23]:
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)
metric_values["model"] = "bce"
features_results.append(metric_values)

In [24]:
recos

,user_id,item_id,score,rank
0,73446,9728,4.363419,1
1,73446,7829,4.254728,2
2,73446,3784,4.122978,3
3,73446,7793,4.038399,4
4,73446,12965,3.816807,5
...,...,...,...,...
947045,857162,12995,2.796035,6
947046,857162,4436,2.643038,7
947047,857162,1844,2.610679,8
947048,857162,14703,2.558943,9


## gBCE loss

In [26]:
RANDOM_SEED = 32
torch.use_deterministic_algorithms(True)
seed_everything(RANDOM_SEED, workers=True)

Seed set to 32


32

In [27]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    loss="gBCE",
    n_negatives=256,
    gbce_t=0.75,
    item_net_block_types=(IdEmbeddingsItemNet, )  # Use only item ids in ItemNetBlock
)

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [28]:
%%time
model.fit(dataset_no_features)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params
---------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 927 K 
---------------------------------------------------------------
927 K     Trainable params
0         Non-trainable params
927 K     Total params
3.709     Total estimated model params size (MB)
/data/home/maspirina1/tasks/repo/RecTools/.venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=143` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: user 3h 3min 59s, sys: 1min 32s, total: 3h 5min 31s
Wall time: 15min 15s


In [29]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_no_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)


/data/home/maspirina1/tasks/repo/RecTools/rectools/models/nn/transformer_data_preparator.py:271: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/maspirina1/tasks/repo/RecTools/rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/data/home/maspirina1/tasks/repo/RecTools/.venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `nu

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 37.6 s, sys: 8.44 s, total: 46 s
Wall time: 32.7 s


In [30]:
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)
metric_values["model"] = "gBCE"
features_results.append(metric_values)

## Softmax with key_padding_mask + causal_mask

In [32]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    item_net_block_types=(IdEmbeddingsItemNet, ),  # Use only item ids in ItemNetBlock
    recommend_device="cuda",
    use_key_padding_mask=True,
)


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [33]:
%%time
model.fit(dataset_no_features)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params
---------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 927 K 
---------------------------------------------------------------
927 K     Trainable params
0         Non-trainable params
927 K     Total params
3.709     Total estimated model params size (MB)
/data/home/maspirina1/tasks/repo/RecTools/.venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=143` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: user 6min 56s, sys: 15.8 s, total: 7min 12s
Wall time: 6min 57s


In [34]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_no_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)

/data/home/maspirina1/tasks/repo/RecTools/rectools/models/nn/transformer_data_preparator.py:271: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/maspirina1/tasks/repo/RecTools/rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/data/home/maspirina1/tasks/repo/RecTools/.venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `nu

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 30.1 s, sys: 3.78 s, total: 33.8 s
Wall time: 24.5 s


In [35]:
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)

In [36]:
metric_values["model"] = "softmax_padding_mask"
features_results.append(metric_values)

In [37]:
features_df = (
    pd.DataFrame(features_results)
    .set_index("model")
    .sort_values(by=["MAP@10", "Serendipity@10"], ascending=False)
)
features_df

,MAP@1,MAP@5,MAP@10,MIUF@1,MIUF@5,MIUF@10,Serendipity@1,Serendipity@5,Serendipity@10
model,,,,,,,,,
softmax,0.049143,0.082793,0.091811,3.672515,4.330234,4.916948,0.000878,0.000723,0.000656
gBCE,0.048091,0.081488,0.090228,3.316945,3.946502,4.641328,0.000644,0.000532,0.000510
softmax_padding_mask,0.046905,0.080541,0.089952,3.772394,4.337196,4.917088,0.000932,0.000723,0.000668
bce,0.041767,0.071836,0.080688,3.813615,4.479653,5.022136,0.000503,0.000534,0.000536


### sasrec with item ids embeddings in ItemNetBlock

In [54]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    item_net_block_types=(IdEmbeddingsItemNet, )  # Use only item ids in ItemNetBlock
)

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [39]:
%%time
model.fit(dataset_no_features)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params
---------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 927 K 
---------------------------------------------------------------
927 K     Trainable params
0         Non-trainable params
927 K     Total params
3.709     Total estimated model params size (MB)
/data/home/maspirina1/tasks/repo/RecTools/.venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=143` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: user 5min 16s, sys: 9.02 s, total: 5min 25s
Wall time: 5min 11s


In [40]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_no_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)

/data/home/maspirina1/tasks/repo/RecTools/rectools/models/nn/transformer_data_preparator.py:271: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/maspirina1/tasks/repo/RecTools/rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/data/home/maspirina1/tasks/repo/RecTools/.venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=143` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 26.9 s, sys: 3.91 s, total: 30.9 s
Wall time: 21.2 s


In [41]:
# TODO: drop `apply(str)`
recos["item_id"] = recos["item_id"].apply(str)
test["item_id"] = test["item_id"].astype(str)
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)
metric_values["model"] = "sasrec_ids"
features_results.append(metric_values)

### sasrec with item ids and category features embeddings in ItemNetBlock

model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    item_net_block_types=(IdEmbeddingsItemNet, CatFeaturesItemNet)  # Use item ids and cat features in ItemNetBlock
)

In [55]:
#%%time
model.fit(dataset_item_features)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params
---------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 927 K 
---------------------------------------------------------------
927 K     Trainable params
0         Non-trainable params
927 K     Total params
3.709     Total estimated model params size (MB)
/data/home/maspirina1/tasks/repo/RecTools/.venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=143` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [56]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_item_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)

/data/home/maspirina1/tasks/repo/RecTools/rectools/models/nn/transformer_data_preparator.py:271: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/maspirina1/tasks/repo/RecTools/rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/data/home/maspirina1/tasks/repo/RecTools/.venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `nu

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 26.9 s, sys: 3.15 s, total: 30 s
Wall time: 21 s


In [57]:
# TODO: drop `apply(str)`
recos["item_id"] = recos["item_id"].apply(str)
test["item_id"] = test["item_id"].astype(str)
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)
metric_values["model"] = "sasrec_ids_cat"
features_results.append(metric_values)

### sasrec with category item features embeddings in ItemNetBlock

In [44]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    item_net_block_types=(CatFeaturesItemNet, )  # Use only cat item features in ItemNetBlock
)

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [45]:
#%%time
model.fit(dataset_item_features)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params
---------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 211 K 
---------------------------------------------------------------
211 K     Trainable params
0         Non-trainable params
211 K     Total params
0.847     Total estimated model params size (MB)
/data/home/maspirina1/tasks/repo/RecTools/.venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=143` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

/data/home/maspirina1/tasks/repo/RecTools/rectools/dataset/features.py:424: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting sparse features to dense array may cause MemoryError")
`Trainer.fit` stopped: `max_epochs=5` reached.


In [46]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_item_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)

/data/home/maspirina1/tasks/repo/RecTools/rectools/models/nn/transformer_data_preparator.py:271: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/maspirina1/tasks/repo/RecTools/rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/data/home/maspirina1/tasks/repo/RecTools/.venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `nu

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 26.2 s, sys: 2.88 s, total: 29 s
Wall time: 20.4 s


In [47]:
# TODO: drop `apply(str)`
recos["item_id"] = recos["item_id"].apply(str)
test["item_id"] = test["item_id"].astype(str)
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)
metric_values["model"] = "sasrec_cat"
features_results.append(metric_values)

In [58]:
features_results

[{'MAP@1': 0.048488013974905805,
  'MAP@5': 0.0821411524595932,
  'MAP@10': 0.0912577746921091,
  'MIUF@1': 3.868676865340589,
  'MIUF@5': 4.506241791317061,
  'MIUF@10': 5.087812416018942,
  'Serendipity@1': 0.0010587683094952943,
  'Serendipity@5': 0.0008279085147448243,
  'Serendipity@10': 0.0007506236395264775,
  'model': 'softmax'},
 {'MAP@1': 0.043093656973408605,
  'MAP@5': 0.07398385923675324,
  'MAP@10': 0.08294397368660968,
  'MIUF@1': 3.598352040478207,
  'MIUF@5': 4.304012747602231,
  'MIUF@10': 4.916007085918255,
  'Serendipity@1': 0.0004767361940337927,
  'Serendipity@5': 0.0004863863900258476,
  'Serendipity@10': 0.000501558401864858,
  'model': 'bce'},
 {'MAP@1': 0.046172701078265196,
  'MAP@5': 0.08031691396493008,
  'MAP@10': 0.08918565294041045,
  'MIUF@1': 3.1867943881001892,
  'MIUF@5': 3.822622559312681,
  'MIUF@10': 4.538795926216838,
  'Serendipity@1': 0.0006401436142825035,
  'Serendipity@5': 0.0005169224313850365,
  'Serendipity@10': 0.0005046814909982423,
  '

### Item to item

In [49]:
target_items = [13865, 4457, 15297]

In [50]:
%%time
recos = model.recommend_to_items(
    target_items=target_items, 
    dataset=dataset_no_features,
    k=10,
    filter_itself=True,
    items_to_recommend=None, #white_list,
)

CPU times: user 1.47 s, sys: 253 ms, total: 1.72 s
Wall time: 1.15 s


In [51]:
recos

,target_item_id,item_id,score,rank
0,13865,15648,1.000000,1
1,13865,3386,1.000000,2
2,13865,16194,0.892315,3
3,13865,147,0.892315,4
4,13865,12586,0.892315,5
5,13865,12309,0.892315,6
6,13865,6661,0.892315,7
7,13865,2255,0.892315,8
8,13865,4130,0.892315,9
9,13865,9109,0.892315,10


In [52]:
# TODO: change model for recos (here is the last one trained and is is the worst in quality)
recos.merge(items[["item_id", "title"]], on="item_id")

,target_item_id,item_id,score,rank,title
0,13865,15648,1.000000,1,Черное золото
1,13865,3386,1.000000,2,Спартак
2,13865,16194,0.892315,3,Голубая линия
3,13865,147,0.892315,4,Единичка
4,13865,12586,0.892315,5,Вспоминая 1942
5,13865,12309,0.892315,6,Враг у ворот
6,13865,6661,0.892315,7,Солдатик
7,13865,2255,0.892315,8,Пленный
8,13865,4130,0.892315,9,Пустота
9,13865,9109,0.892315,10,Последняя битва
